<a href="https://colab.research.google.com/github/kavyajeetbora/document_word_search/blob/main/pdf_word_scanner_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDF Word Scanner V 2.0

## Install dependencies and Import libraries

In [ ]:
## Pdf libraries
%%time

!sudo apt-get update -y
!sudo apt-get install -y poppler-utils
!pip install pdf2image
!pip install pytesseract
!sudo apt install tesseract-ocr

from pdf2image import convert_from_path

import pandas as pd
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk import word_tokenize

from PIL import Image
import pytesseract

from tqdm.notebook import tqdm

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.0 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://security.ubuntu.com/ubu

In [ ]:
def search_word_sentence(search_word, text):
    '''
    Returns the sentences where the keyword is found given a text data
    '''
    imp_sentences = []
    text = text.lower()
    sentences = sent_tokenize(text)
    for sent in sentences:
        if search_word in sent:
            imp_sentences.append(sent)
    return imp_sentences

def convert_pdf_to_text(pdf_file_path,image_ppi=100):
    '''
    This function converts the pdf into text per page
    and returns an array of textual data
    '''
    texts = []
    images = convert_from_path(pdf_file_path, image_ppi)
    for image in tqdm(images, total=len(images), unit="Page"):
        texts.append(pytesseract.image_to_string(image))
    return texts

def search_word_in_PDF(key_word, textual_data):
    '''
    Returns a dataframe containing the sentences in which the key word is found 
    from the given textual data for each page
    '''
    Pages = []
    key_word = key_word.lower()
    for i,text in tqdm(enumerate(textual_data),total=len(textual_data),unit="page"):
        sentences = search_word_sentence(key_word,text)
        if len(sentences) > 0:
            for sent in sentences:
                Pages.append([i+1,sent])
    return pd.DataFrame(Pages, columns=['Page Number','sentence'])

## Convert

1. Convert the pdf into list of images
2. From the images, exctract the text

### Upload the PDF file

In [ ]:
%%time
from google.colab import files
pdf_file = files.upload()

filename = list(pdf_file.keys())[0]
print("successfully uploaded",filename)

Saving CV of Kavyajeet Bora.pdf to CV of Kavyajeet Bora.pdf
successfully uploaded CV of Kavyajeet Bora.pdf
CPU times: user 170 ms, sys: 24.2 ms, total: 195 ms
Wall time: 11.4 s


### Convert the PDF into Textual Data

In [ ]:
%%time
if filename.endswith(".pdf"):
    textual_data = convert_pdf_to_text(filename, image_ppi=100)
else:
    print("Not a pdf file")

  0%|          | 0/5 [00:00<?, ?Page/s]

CPU times: user 215 ms, sys: 120 ms, total: 335 ms
Wall time: 20.8 s


**search the keyword**

In [ ]:
%%time

#@title String fields
keyword = "geotechnical" #@param {type:"string"}

data = search_word_in_PDF(keyword, textual_data)
if len(data) == 0:
    print("No Keyword: {0} found".format("india"))
else:
    print("{0} Number of sentences found".format(len(data)))

  0%|          | 0/5 [00:00<?, ?page/s]

11 Number of sentences found
CPU times: user 77.6 ms, sys: 6.43 ms, total: 84 ms
Wall time: 99.8 ms


## Output and export the sentences into excel

In [ ]:
## output the file
data

,Page Number,sentence
0,1,"career\n2021-present\ndata analyst, ramboll\nr..."
1,1,itd\n\nworked as a geotechnical design enginee...
2,1,ltd\nworked as a technical sales engineer\n\ne...
3,3,"1, data visualization and data summarization o..."
4,3,it is 23.3-km elevated metro line with\n23 sta...
5,3,expected to reduce travel time between south m...
6,3,task performed:\n\n1. prepared the geotechnica...
7,4,in this project i worked on the following -\n\...
8,4,"this,\nproject included construction of 3 unde..."
9,4,2016-2017\n\nprojects in maccaferri\n\nworked ...


In [ ]:
## export it to excel and download
data.to_excel("search.xlsx")
files.download('search.xlsx') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>